In [1]:
import os
import spender
import torch
from accelerate import Accelerator

# get some SDSS spectra from the ids, store locally in data_path
data_dir = r"C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\data"

ids = []

for plate in os.listdir(data_dir):
    if '.git' not in plate: 
        plate_dir = os.path.join(data_dir, plate)
        for file in os.listdir(plate_dir):

            if '.git' not in file: 

                #if '.fits' not in file: 
                #    os.rename(os.path.join(plate_dir, file), os.path.join(plate_dir, f'{file}.fits'))

                ids.append([int(i) for i in file.split('.')[0].split('-')[1:]])

for i in ids: 
    print(i)

c:\Users\tkiker\AppData\Local\miniconda3\envs\agn310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[10227, 58224, 324]
[10227, 58224, 591]
[10228, 58223, 888]
[10229, 58441, 82]
[10229, 58441, 578]
[10229, 58441, 718]
[10231, 58171, 384]
[10231, 58171, 649]
[10232, 58223, 123]
[10232, 58223, 146]
[10232, 58223, 175]
[10232, 58223, 682]
[10233, 58130, 358]
[10233, 58130, 795]
[10234, 58153, 407]
[10234, 58153, 540]
[10237, 58154, 166]
[10237, 58154, 247]
[10237, 58154, 515]
[10238, 58155, 206]
[10238, 58155, 573]
[10238, 58155, 977]
[10239, 58142, 21]
[10239, 58142, 262]
[10239, 58142, 352]
[10239, 58142, 945]
[10240, 58158, 111]
[10241, 58157, 7]
[10241, 58157, 354]
[10241, 58157, 358]
[10241, 58157, 496]
[10241, 58157, 608]
[10241, 58157, 788]
[10242, 58161, 132]
[10242, 58161, 321]
[10242, 58161, 378]
[10243, 58159, 206]
[10243, 58159, 352]
[10243, 58159, 379]
[10244, 58225, 60]
[10244, 58225, 74]
[10244, 58225, 352]
[10244, 58225, 608]
[10245, 58161, 394]
[10247, 58162, 218]
[10247, 58162, 694]
[10247, 58162, 834]
[10247, 58162, 942]
[10248, 58220, 245]
[10248, 58220, 280]
[10248

In [4]:
import numpy as np
import sklearn 
import sklearn.model_selection
from spender.data.sdss import BOSS

data_dir = r"C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\data"
batch_dir = r'C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\batched-data'

boss = BOSS()

shuffled_ids = ids[0:1056]
np.random.shuffle(shuffled_ids)

variable_indices = [[i*40, (i+1)*40] for i in range(0, 25)]

for counter, index_range in enumerate(variable_indices): 
    spec, w, z, norm, zerr = BOSS.make_batch(data_dir, shuffled_ids[index_range[0]:index_range[1]])
    BOSS.save_batch(dir=batch_dir, batch=[spec, w, z, norm, zerr], tag='variable', counter=counter)

In [ ]:
# hardware optimization
accelerator = Accelerator(mixed_precision='fp16')

In [14]:
from astropy.io import fits


filename = r'C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\data\3590\spec-3590-55201-0172.fits'
hdulist = fits.open(filename, ignore_missing_simple=True)
data = hdulist[1].data
data

        Use textwrap.indent() instead. [astropy.io.fits.hdu.hdulist]
    Header size is not multiple of 2880: 126107
There may be extra bytes after the last HDU or the file is corrupted. [astropy.io.fits.hdu.hdulist]


OSError: Empty or corrupt FITS file

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(self, input_dim):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv1d(1, 128, kernel_size=5, padding='same')
        self.prelu1 = nn.PReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)

        self.conv2 = nn.Conv1d(128, 256, kernel_size=11, padding='same')
        self.prelu2 = nn.PReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2)

        self.conv3 = nn.Conv1d(256, 512, kernel_size=21, padding='same')
        self.prelu3 = nn.PReLU()
        self.pool3 = nn.MaxPool1d(kernel_size=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(512 * (input_dim // 8), 256)
        self.prelu4 = nn.PReLU()
        self.fc2 = nn.Linear(256, 128)
        self.prelu5 = nn.PReLU()
        self.fc3 = nn.Linear(128, 64)
        self.prelu6 = nn.PReLU()

        self.latent_space = nn.Linear(64, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.prelu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.prelu2(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = self.prelu3(x)
        x = self.pool3(x)

        x = self.flatten(x)

        x = self.fc1(x)
        x = self.prelu4(x)
        x = self.fc2(x)
        x = self.prelu5(x)
        x = self.fc3(x)
        x = self.prelu6(x)

        x = self.latent_space(x)

        return x

class Decoder(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim, 64)
        self.prelu1 = nn.PReLU()
        self.fc2 = nn.Linear(64, 128)
        self.prelu2 = nn.PReLU()
        self.fc3 = nn.Linear(128, 256)
        self.prelu3 = nn.PReLU()
        self.fc4 = nn.Linear(256, 512 * (output_dim // 8))
        self.prelu4 = nn.PReLU()

        self.unflatten = nn.Unflatten(1, (512, output_dim // 8))

    def forward(self, x):
        x = self.fc1(x)
        x = self.prelu1(x)
        x = self.fc2(x)
        x = self.prelu2(x)
        x = self.fc3(x)
        x = self.prelu3(x)
        x = self.fc4(x)
        x = self.prelu4(x)

        x = self.unflatten(x)

        return x

class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(input_dim)
        self.decoder = Decoder(latent_dim=10, output_dim=input_dim)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Define input dimension (e.g., 3892 spectral elements)
input_dim = 3892

# Create the autoencoder model
model = Autoencoder(input_dim=input_dim)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Print model summary (requires torchsummary package)
from torchsummary import summary
summary(model, (1, input_dim))

In [ ]:
# make data batch for tensorflow testing! 
import os 
from tqdm import tqdm 
import matplotlib.pyplot as plt 
import smplotlib
from astropy.io import fits 
import numpy as np
import scipy 

spectra_IDs = []
normalized_spectra = []

root_dir = r'C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\data'
plot_dir = r'C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\code\plots'


for plate in tqdm(os.listdir(root_dir)):
    if 'git' not in plate: 
        plate_dir = os.path.join(root_dir, plate)
        for filename in os.listdir(plate_dir):
            file_path = os.path.join(plate_dir, filename)

            hdul = fits.open(file_path)

            z = hdul[2].data['z'][0]

            if z <=3.15 and z>=2.15: 

                data = hdul[1].data

                wavelength = 10**data['loglam']
                flux = data['flux']
                ivar = data['ivar']

                quality_mask = ivar>0 

                wavelength = wavelength[quality_mask]
                flux = flux[quality_mask]

                wavelength /= (1+z)

                normalizing_mask = np.logical_and(wavelength>=2000, wavelength<=2400)
                normalizing_median = np.median(flux[normalizing_mask])
                normalized_flux = flux / normalizing_median

                median_filtered = scipy.signal.medfilt(normalized_flux, 7)

                x = np.arange(1400, 2400, 1)
                y = np.interp(x, wavelength, median_filtered)

                fig, ax = plt.subplots(figsize=(4,4))

                ax.plot(wavelength, scipy.signal.medfilt(normalized_flux, 7), color='black', label=f'z={str(round(z, 2))}')
                ax.plot(x, y, color='red')
                ax.set_xlabel('Rest Wavelength (A)')
                ax.set_ylabel('Normalized and Smoothed Flux')
                ax.set_title(filename)
                plt.savefig(os.path.join(plot_dir, filename.replace('fits', 'png')))

                plt.clf()

                spectrum_ID = '-'.join(filename.split('-')[1:])[:-5]
                spectra_IDs.append(spectrum_ID)
                normalized_spectra.append(y)

In [31]:
np.array(normalized_spectra).shape

(3185, 1000)

In [33]:
import pandas as pd 
spectra_df = pd.DataFrame()
for spectrum_ID, normalized_spectrum in tqdm(zip(spectra_IDs, normalized_spectra)): 
    spectra_df[spectrum_ID] = normalized_spectrum

spectra_df.to_csv(r'C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\code\normalized_spectra.csv', index=False)

3185it [00:03, 822.16it/s] 


In [34]:
spectra_df

,10443-58155-0261,10450-58142-0643,10458-58143-0522,10474-58143-0478,10474-58143-0846,10475-58137-0042,10477-58136-0521,10477-58136-0903,10478-58171-0468,10760-58224-0600,...,4743-55645-0713,4743-55645-0806,4743-55645-0940,4746-55632-0026,4746-55632-0200,4746-55632-0276,4746-55632-0518,4746-55632-0540,4746-55632-0882,4746-55632-0924
0,1.548299,1.400521,1.170374,1.481776,1.379109,2.263475,1.291375,1.880707,1.571029,2.308350,...,2.222785,1.989938,1.579391,2.068434,2.013907,2.582308,1.513710,2.116657,1.774880,2.199803
1,1.539278,1.420687,1.202627,1.426070,1.340575,2.291186,1.305914,1.790940,1.576424,2.293618,...,2.211879,1.986896,1.537143,2.068434,1.947338,2.544816,1.586896,2.110583,1.840842,2.171098
2,1.576039,1.257479,1.229691,1.510434,1.367838,2.180172,1.264553,1.894164,1.641587,2.198425,...,2.221303,1.991258,1.394587,2.077104,1.886011,2.529104,1.509998,2.054212,1.867922,2.077917
3,1.562930,1.190799,1.198062,1.511236,1.397491,2.219530,1.329778,1.835259,1.600572,2.032003,...,2.205795,1.969351,1.356737,1.984402,1.806138,2.438583,1.506585,1.962477,1.867922,2.077917
4,1.478465,1.158222,1.153444,1.480687,1.421197,2.339613,1.352435,1.759009,1.661734,2.078477,...,2.032104,1.916237,1.404794,1.913324,1.678650,2.525343,1.506585,2.078305,1.812177,2.112257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.842234,0.786117,0.900414,0.869274,0.646558,0.855508,0.933958,0.830181,0.825429,0.941833,...,0.947488,1.073671,1.117268,0.954697,0.972372,0.986286,1.063580,0.922361,1.008158,0.945010
996,0.846138,0.780119,0.900248,0.872914,0.640589,0.863753,0.950092,0.824011,0.805100,0.941623,...,0.941976,1.069730,1.137914,0.968645,0.925410,0.833241,1.066752,0.855778,1.178476,0.953580
997,0.859528,0.780525,0.900248,0.866256,0.641222,0.802459,0.867102,0.839157,0.812730,0.967683,...,0.956770,1.065084,1.140025,0.945289,0.976933,0.705335,1.046531,0.922361,1.028551,0.945204
998,0.867921,0.802993,0.915306,0.863236,0.644911,0.863753,0.852557,0.829424,0.818648,0.950040,...,0.985554,1.041150,1.125993,0.936218,0.967397,0.739780,1.030636,1.015243,1.028551,0.945107


In [29]:
import shutil 
from tqdm import tqdm 

destiation_dir = r'C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\code\data-export'
root_dir = r'C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\data'

for spectra_ID in tqdm(spectra_IDs): 
    plate = spectra_ID.split('-')[0]
    plate_dir = os.path.join(root_dir, plate)
    filename = f'spec-{spectra_ID}.fits'

    origin_path = os.path.join(plate_dir, filename)
    destination_path = os.path.join(destiation_dir, filename)

    shutil.copyfile(origin_path, destination_path)

10443
C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\data\10443
C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\data\10443\spec-10443-58155-0261.fits C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\code\data-export\spec-10443-58155-0261.fits
10450
C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\data\10450
C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\data\10450\spec-10450-58142-0643.fits C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\code\data-export\spec-10450-58142-0643.fits
10458
C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\data\10458
C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\data\10458\spec-10458-58143-0522.fits C:\Users\tkiker\Documents\GitHub\AGN-UMAP\code\monthly\july2024\wk4\sdss\code\data-export\spec-10458-58143-0522.fits
10474
C:\Users\tkiker\Documents\GitHub\AGN-UMA

KeyboardInterrupt: 